# Tratamento de Lista Geral de Leads

### Objetivo: Esse pipeline tem como objetivo separar a lista de Leads disponível para disparos e consumo

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe,
    remover_leads_do_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


## Load Leads Dataframe

In [2]:
df = db.execute_query_from_file("select_complete_lead_data_from_conversions_1-3")
ofan_leads = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"conversion_type_id": 1, "campaign_id": "lcto-ofan-jan26"})

In [3]:
## Filter Buyers from Dataframe
df = remove_buyers_from_dataframe(df)

df = remover_leads_do_dataframe(df, ofan_leads)

✓ Removidos 13,527 compradores de 302,965 registros (4.5% filtrado)
✓ Removidos 21,318 leads de 289,438 registros (7.4% filtrado)


In [4]:
# garante datetime
df["last_conversion_date"] = pd.to_datetime(df["last_conversion_date"], format="%Y-%m-%d", errors="coerce")

# quarter no formato "24-Q1"
df["quarter"] = (
    df["last_conversion_date"].dt.strftime("%y")
    + "-Q"
    + df["last_conversion_date"].dt.quarter.astype("string")
)

In [5]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Filtra linhas onde 'quarter' contém "Q3" ou "Q4"
    df = df[
        df['quarter'].str.contains("Q1", regex=False, na=False, case=False) |
        df['quarter'].str.contains("Q2", regex=False, na=False, case=False) |
        df['quarter'].str.contains("Q3", regex=False, na=False, case=False) |
        df['quarter'].str.contains("Q4", regex=False, na=False, case=False)
    ]
    # Filtra linhas onde 'quarter' contém "25-"
    df = df[df['quarter'].str.contains("25-", regex=False, na=False, case=False)]
    # Remove linhas sem valor em 'phone'
    df = df.dropna(subset=['phone'])
    return df

df_clean = clean_data(df.copy())

# Usa parênteses para garantir precedência correta dos operadores bitwise
df_qualified = df_clean[(df_clean['qualificado_geral'] == True) | (df_clean['renda_qualificada'] == True)]
df_unqualified = df_clean[(df_clean['qualificado_geral'] == False) & (df_clean['renda_qualificada'] == False)]

save_processed_data(df_qualified, "ofan26_qualified_list.csv", data_dir='../data/processed')
save_processed_data(df_unqualified, "ofan26_unqualified_list.csv", data_dir='../data/processed')
save_processed_data(df_clean, "ofan26_full_list.csv", data_dir='../data/processed')